# Messungen mit JupyterLab auf dem STEMlab

Wenn Sie erfolgreich ein neues Jupyter-Notebook erstellt haben, kopieren Sie den untenstehenden Code in eine Zelle und
führen diese dann aus. Der untenstehende Code erzeugt das gleiche Signal wie in @fig-22-05, aber er zeichnet sie in
einam XY-Diagramm auf, wobei die x-Achse die Diodenspannung $IN_2$ und die y-Achse den Diodenstrom $I_D = I_C = (IN_1 -
IN_2) / R_3$ darstellt.


In [ ]:
# Import libraries
from redpitaya.overlay.mercury import mercury as overlay

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d, LinearAxis, LabelSet, Label
from bokeh.plotting import figure, output_file, show
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

import numpy as np

# Initialize fpga modules
fpga = overlay()
gen0 = fpga.gen(0)
osc = [fpga.osc(ch, 1.0) for ch in range(fpga._MNO)]

# Configure OUT1 generator channel
gen0.amplitude = 0.8
gen0.offset = -0.12
gen0.waveform = gen0.sawtooth(0.5)
gen0.frequency = 2000
gen0.start()
gen0.enable = True
gen0.trigger()

# R1 resistor value
R1 = 1000

# Configure IN1 and IN2 oscilloscope input channels
for ch in osc:
    ch.filter_bypass = True

    # data rate decimation
    ch.decimation = 10

    # trigger timing [sample periods]
    N = ch.buffer_size
    ch.trigger_pre = 0
    ch.trigger_post = N

    # osc0 is controlling both channels
    ch.sync_src = fpga.sync_src["osc0"]
    ch.trig_src = fpga.trig_src["osc0"]

    # trigger level [V], edge ['neg', 'pos'] and holdoff time [sample periods]
    ch.level = 0.01
    ch.edg = 'pos'
    ch.holdoff = 0

    # Initialize diode current and voltage
    V = I = np.zeros(N)

    # Plotting
    hover = HoverTool(mode='vline', tooltips=[("V", "@x"), ("I", "@y")])
    tools = "wheel_zoom, box_zoom, reset,pan"
    p = figure(plot_height=500,
               plot_width=900,
               title="XY plot of transistor VI characteristic",
               toolbar_location="right",
               tools=(tools, hover))
    p.xaxis.axis_label = 'Voltage [V]'
    p.yaxis.axis_label = 'Current [mA]'
    r = p.line(V, I, line_width=1, line_alpha=0.7, color="blue")

    # get and explicit handle to update the next show cell
    target = show(p, notebook_handle=True)

Erstellen Sie eine neue Zelle (Einfügen -> Zelle darunter) und kopieren Sie den Code von unten hinein.


In [ ]:
# Measuring I, V  and re-plotting
while True:
    # reset and start
    osc[0].reset()
    osc[0].start()

    # wait for data
    while (osc[0].status_run()):
        pass
    V0 = osc[0].data(N-100)*10  # IN1 signal
    V1 = osc[1].data(N-100)*10  # IN2 signal
    I = ((V0-V1)/R1)*1E3        # 1E3 convert to mA
    r.data_source.data['x'] = V0
    r.data_source.data['y'] = I

    push_notebook(handle=target)

Führen Sie Zelle 1 und Zelle 2 aus. Zelle 2 ist eine Hauptschleife für die Erfassung und Neuaufnahme. Wenn Sie
die Erfassung stoppen, führen Sie einfach nur Zelle 2 aus, um die Messungen erneut zu starten.

Nach dem Ausführen des obigen Codes sollten Sie die IV-Charakteristik, wie in @fig-22-05 gezeigt, erhalten. 

![BJT IV-Charakteristik gemessen mit Jupyter Notebook.](../images/Activity_22_Fig_05.png){#fig-22-05}

In @fig-22-05 ist die in einer Diodenkonfiguration gemessene IV-Charakteristik eines BJT dargestellt.

Vergleichen sie diese Ergebnisse mit der IV-Charakteristik der pn-Diode. Es sollte eine Hysterese sichtbar sein.
Überlegen sie, warum Bipolartransistoren als Dioden verwendet werden können.